<a href="https://colab.research.google.com/github/EN10/modded-llama2.c/blob/main/Baby_Llama_128.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%time
# download training and inference code
!git clone https://github.com/eniompw/modded-llama2.c

In [ ]:
%%time
# download and extract tinystories dataset
!. ./modded-llama2.c/download_tinystories.sh

In [ ]:
%%time
# train neural net
!cd modded-llama2.c && python train.py --max_iters=1 #--init_from='resume'

In [ ]:
%%time
# test inference
!cd ./modded-llama2.c && ./run out/model.bin -i "Once upon a time "